### Importing required Libraries

In [1]:
#Importing Libraries
!pip install nltk

import nltk
import pickle
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Data Loading and Preprocessing

In [2]:
#Dataset Loading
dataset = pd.read_csv('spam1.csv',names=['sentiment','statement'])
dataset

,sentiment,statement
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
1313,ham,I'm working technical support voice process.n...
1314,ham,I might come to kerala for 2 days.so you can b...
1315,ham,Ok. Not sure what time tho as not sure if can ...
1316,ham,"That's fine, I'll bitch at you about it later ..."


In [3]:
def cleanString(incomingString):
    newstring = incomingString
    newstring = newstring.replace("!","")
    newstring = newstring.replace("@","")
    newstring = newstring.replace("#","")
    newstring = newstring.replace("$","")
    newstring = newstring.replace("%","")
    newstring = newstring.replace("^","")
    newstring = newstring.replace("&","and")
    newstring = newstring.replace("*","")
    newstring = newstring.replace("(","")
    newstring = newstring.replace(")","")
    newstring = newstring.replace("+","")
    newstring = newstring.replace("=","")
    newstring = newstring.replace("?"," ")
    newstring = newstring.replace("\'","")
    newstring = newstring.replace("\"","")
    newstring = newstring.replace("'","")
    newstring = newstring.replace("'m","am")
    newstring = newstring.replace("}","")
    newstring = newstring.replace("[","")
    newstring = newstring.replace("]","")
    newstring = newstring.replace("<","")
    newstring = newstring.replace(">","")
    newstring = newstring.replace("~","")
    newstring = newstring.replace("`","")
    newstring = newstring.replace(":","")
    newstring = newstring.replace(";","")
    newstring = newstring.replace("|","")
    newstring = newstring.replace("\\","")
    newstring = newstring.replace("/","") 
    newstring = newstring.replace("0","")
    newstring = newstring.replace("1","")
    newstring = newstring.replace("2","")
    newstring = newstring.replace("3","")
    newstring = newstring.replace("4","")
    newstring = newstring.replace("5","")
    newstring = newstring.replace("6","")
    newstring = newstring.replace("7","")
    newstring = newstring.replace("8","")
    newstring = newstring.replace("9","")  
    newstring = newstring.replace(".","")
    newstring = newstring.replace(",","")
    return newstring

In [4]:
clean_text = []
for col,row in dataset.iterrows():
  text = cleanString(row[1])
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
  text = (" ").join(tokens_without_sw)
  clean_text.append(text)
dataset['statement'] = clean_text

### Model training and Pickling

In [5]:
#Training the model and predicting
#Creating Vocabulary
id = dataset['statement']
model = Tokenizer()
model.fit_on_texts(list(id))

X = dataset['statement']
y = dataset['sentiment']


X = model.texts_to_matrix(X, mode='count')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print((X_test.shape[0], (y_test == y_pred).sum()))

(264, 224)


In [6]:
#Creating the classifier function for queries
def classifier(text):
  text_ = cleanString(text)
  text_tokens = word_tokenize(text_)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
  text = (" ").join(tokens_without_sw)
  vec = model.texts_to_matrix([text], mode='count')
  return gnb.predict(vec)

In [7]:
print(classifier("Sorry, I'll call later"))

['ham']


In [9]:
print(classifier("Congratulation! You won a Prize!"))

['spam']


In [8]:
#Serialization the classifier using pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()